# Data Processing

Full data dictionary available [here](https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2018/cc-est2018-alldata.pdf)

%%html
<style>
table {float:left}
</style>

|Code|Demographic|
|:---|:---|
|AAC_MALE|Asian alone or in combination male population|
|AAC_FEMALE|Asian alone or in combination female population|

|YEAR Code|Dates|
|:---|:---|
|1|4/1/2010 Census population|
|2|4/1/2010 population estimates base|
|3|7/1/2010 population estimate|
|4|7/1/2011 population estimate|
|5|7/1/2012 population estimate|
|6|7/1/2013 population estimate|
|7|7/1/2014 population estimate|
|8|7/1/2015 population estimate|
|9|7/1/2016 population estimate|
|10|7/1/2017 population estimate|
|11|7/1/2018 population estimate|

|AGEGRP Code|Age|
|:---|:---|
|0|Total|
|1|Age 0 to 4 years|
|2|Age 5 to 9 years|
|3|Age 10 to 14 years|
|4|Age 15 to 19 years|
|5|Age 20 to 24 years|
|6|Age 25 to 29 years|
|7|Age 30 to 34 years|
|8|Age 35 to 39 years|
|9|Age 40 to 44 years|
|10|Age 45 to 49 years|
|11|Age 50 to 54 years|
|12|Age 55 to 59 years|
|13|Age 60 to 64 years|
|14|Age 65 to 69 years|
|15|Age 70 to 74 years|
|16|Age 75 to 79 years|
|17|Age 80 to 84 years|
|18|Age 85 years or older|

In [1]:
import pandas as pd
import numpy as np

### Read in data

In [2]:
df = pd.read_csv("../data/cc-est2018-alldata-36.csv")

In [3]:
df.head()

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,...,HWAC_MALE,HWAC_FEMALE,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE
0,50,36,1,New York,Albany County,1,0,304204,147076,157128,...,5236,5416,2125,2295,361,410,139,141,41,41
1,50,36,1,New York,Albany County,1,1,15286,7821,7465,...,488,514,288,277,44,40,18,12,5,3
2,50,36,1,New York,Albany County,1,2,16131,8224,7907,...,452,443,275,258,31,36,15,11,5,3
3,50,36,1,New York,Albany County,1,3,17639,9065,8574,...,448,435,230,204,33,28,13,12,2,1
4,50,36,1,New York,Albany County,1,4,23752,11925,11827,...,632,627,271,283,50,46,16,15,4,6


In [5]:
df["CTYNAME"].unique()

array(['Albany County', 'Allegany County', 'Bronx County',
       'Broome County', 'Cattaraugus County', 'Cayuga County',
       'Chautauqua County', 'Chemung County', 'Chenango County',
       'Clinton County', 'Columbia County', 'Cortland County',
       'Delaware County', 'Dutchess County', 'Erie County',
       'Essex County', 'Franklin County', 'Fulton County',
       'Genesee County', 'Greene County', 'Hamilton County',
       'Herkimer County', 'Jefferson County', 'Kings County',
       'Lewis County', 'Livingston County', 'Madison County',
       'Monroe County', 'Montgomery County', 'Nassau County',
       'New York County', 'Niagara County', 'Oneida County',
       'Onondaga County', 'Ontario County', 'Orange County',
       'Orleans County', 'Oswego County', 'Otsego County',
       'Putnam County', 'Queens County', 'Rensselaer County',
       'Richmond County', 'Rockland County', 'St. Lawrence County',
       'Saratoga County', 'Schenectady County', 'Schoharie County',
     